In [6]:
import pandas as pd

import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import joblib


%matplotlib inline

In [7]:
def trim_set(frame):
    return frame[((frame['TRB'] > 2) | (frame['AST'] > 5)) & (frame['PTS'] > 15)]
    # return frame[((frame['TRB'] > 5) | (frame['AST'] > 5)) & (frame['PTS'] > 15) & (frame['Win%'] > .5)]

In [34]:
df = pd.read_csv('train_set_full.csv', index_col=0)

print(df.columns)

df = df.drop(columns=['STAR', 'DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
                      'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
                      'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
                      'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
                      'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
                      'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
                      'SMOTY'], errors='ignore')

df = trim_set(df)

df['MVP_Rank'] = df['MVP_Rank'].astype("int")



Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'MVP_Rank', 'MVP_First', 'MVP_Pts Won', 'MVP_Pts Max',
       'MVP_Share'],
      dtype='object')


In [35]:
import os
from datetime import datetime

import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [36]:
test_set = df[df['Year'] == 1980]
test_set = trim_set(test_set)
# test_set


y_test = test_set.loc[:,'MVP_Share',]
y_test

0      0.665
7      0.000
15     0.068
16     0.000
23     0.000
24     0.000
39     0.000
55     0.000
56     0.000
58     0.000
59     0.000
66     0.000
70     0.000
74     0.143
79     0.000
81     0.000
85     0.086
87     0.000
92     0.000
101    0.000
113    0.000
118    0.009
119    0.000
121    0.000
122    0.000
123    0.000
132    0.000
138    0.000
148    0.000
150    0.005
151    0.000
153    0.000
159    0.000
166    0.000
167    0.000
171    0.000
172    0.000
185    0.000
190    0.000
192    0.000
195    0.009
196    0.000
200    0.000
202    0.000
206    0.000
207    0.000
215    0.000
216    0.000
233    0.000
234    0.000
239    0.000
241    0.000
242    0.007
243    0.000
248    0.000
Name: MVP_Share, dtype: float64

In [37]:
train_set = df[df['Year'] != 1980]
train_set = trim_set(train_set)

y_train = df.loc[(df['Year'] != 1980),'MVP_Share',]
y_train.head(2)

0    0.414
6    0.000
Name: MVP_Share, dtype: float64

In [38]:

feature_list = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA',
                'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
                'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
                'BPM', 'VORP']

In [39]:
x_train = train_set[feature_list]
x_test = test_set[feature_list]

In [40]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression,SelectKBest,SelectPercentile,chi2

# declare a two step pipeline, explicitly giving names to both steps.
pipe = Pipeline(

    [('feat_selection', SelectPercentile(percentile=60),),('scaler', MinMaxScaler()), ('reg', RandomForestRegressor())])

pipe.fit(x_train, y_train)

Pipeline(steps=[('feat_selection', SelectPercentile(percentile=60)),
                ('scaler', MinMaxScaler()), ('reg', RandomForestRegressor())])

In [41]:
y_pred = pipe.predict(X=x_test)

In [42]:
test_set['pred'] = y_pred
test_set['pred_scaled'] = MinMaxScaler().fit_transform(y_pred.reshape(-1, 1))
test_set['pred_rank'] = test_set.pred.rank(ascending=False).astype("int")


In [59]:
saved = test_set.sort_values('pred_rank', ascending=True).head(10)[
    ['pred_rank','MVP_Rank', 'Player', 'G', 'PTS', 'TS%', 'AST', 'TRB', 'STL', 'WS/48', 'pred']].reset_index()

# correct_list = []
# for i in range (0,5):
    # print(saved['pred_rank'].iloc[i] != saved['MVP_Rank'].iloc[i])
i = 0
print(f'{saved.loc[i,"Player"]}\nPredicted - {saved.loc[i,"pred_rank"]}\nActual - {saved.loc[i,"MVP_Rank"]}\n')
# if saved.loc[i,"pred_rank"] == saved.loc[i,"MVP_Rank"]:
#     correct_list.append(True)


Kareem Abdul-Jabbar
Predicted - 1
Actual - 1



In [ ]:
if not os.path.isdir(f'Historic Predictions/{datetime.now().strftime("%Y")}'):
    os.mkdir(f'Historic Predictions/{datetime.now().strftime("%Y")}')

In [ ]:
saved.to_csv(f'Historic Predictions/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%Y%m%d")}_MVP_Predictions.csv')
print(test_set.sort_values('pred_rank', ascending=True).head(10)[
          ['pred_rank','Player', 'G', 'PTS', 'FG%', 'AST', 'TRB', 'STL', 'pred']])

,index,pred_rank,MVP_Rank,Player,G,PTS,TS%,AST,TRB,STL,WS/48,pred
4,122,5,0,Marques Johnson,77,21.7,0.585,3.5,7.4,1.3,0.206,0.08731
6,81,7,0,World B. Free,68,30.2,0.544,4.2,3.5,1.2,0.148,0.05782
7,151,8,0,Cedric Maxwell,80,16.9,0.679,2.5,8.8,1.0,0.214,0.05579
8,56,9,0,Adrian Dantley,68,28.0,0.635,2.8,7.6,1.4,0.189,0.03715
9,113,10,0,Dan Issel,82,23.8,0.571,2.4,8.8,1.1,0.188,0.03275
0,0,1,1,Kareem Abdul-Jabbar,82,24.8,0.639,4.5,10.8,1.0,0.227,0.40369
1,74,2,2,Julius Erving,78,26.9,0.568,4.6,7.4,2.2,0.213,0.39174
2,85,3,3,George Gervin,78,33.1,0.587,2.6,5.2,1.4,0.173,0.14035
5,242,6,8,Gus Williams,82,22.1,0.529,4.8,3.4,2.4,0.187,0.07763
3,150,4,9,Moses Malone,82,25.8,0.560,1.8,14.5,1.0,0.183,0.11835


In [64]:
saved.loc['pred_rank' == 1,'Player',]

KeyError: 'False: boolean label can not be used without a boolean index'

In [65]:
saved['pred_rank' == 1]

KeyError: False

In [70]:
saved[saved.pred_rank == 1].Player.values[0]

'Kareem Abdul-Jabbar'